## Install all needed packages in one step

In [3]:
!pip install ipykernel debugpy
#!pip install mne scikit-learn numpy ipywidgets
# Uncomment the above line to install dependencies by removing the hast tag (#)


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\mw24396\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import os
import ipywidgets as widgets
from IPython.display import display

# Function to get main folders
def get_main_folders(parent_dir):
    return [
        d for d in os.listdir(parent_dir)
        if os.path.isdir(os.path.join(parent_dir, d)) and d != '.git'
    ]

# Function to check if a folder contains .edf files, recursively
def contains_edf_files(folder_path):
    for root, _, files in os.walk(folder_path):
        if any(f.endswith('.edf') for f in files):
            return True
    return False

# Function to get subfolders with .edf files and proper formatting
def get_subfolders(parent_dir):
    options = []
    for main_folder in get_main_folders(parent_dir):
        main_folder_path = os.path.join(parent_dir, main_folder)
        if contains_edf_files(main_folder_path):  # Check if main folder contains .edf files
            options.append(main_folder)  # Add main folder
            for subfolder in os.listdir(main_folder_path):
                subfolder_path = os.path.join(main_folder_path, subfolder)
                if os.path.isdir(subfolder_path) and subfolder != '.git' and contains_edf_files(subfolder_path):
                    options.append(f"----- {subfolder}")  # Add indented subfolder
                    # Check for nested subfolders
                    for nested_subfolder in os.listdir(subfolder_path):
                        nested_subfolder_path = os.path.join(subfolder_path, nested_subfolder)
                        if os.path.isdir(nested_subfolder_path) and nested_subfolder != '.git' and contains_edf_files(nested_subfolder_path):
                            options.append(f"-------- {nested_subfolder}")  # Add further indented subfolder
    return options

# Function to handle the dropdown change
def on_directory_selected(change):
    selected_directory = change['new']
    if selected_directory:
        selected_path = None
        if selected_directory.startswith("----- "):
            subfolder_name = selected_directory.replace("----- ", "")
            main_folder = next(
                (folder for folder in get_main_folders(parent_directory)
                 if os.path.isdir(os.path.join(parent_directory, folder, subfolder_name)) and contains_edf_files(os.path.join(parent_directory, folder, subfolder_name))),
                None
            )
            if main_folder:
                selected_path = os.path.join(parent_directory, main_folder, subfolder_name)
        else:
            selected_path = os.path.join(parent_directory, selected_directory)

        print(f'Selected directory: {selected_path}')
        return selected_path

# Function to open the selected folder
def open_folder(b):
    selected_path = on_directory_selected({'new': directory_dropdown.value})
    if selected_path:
        os.startfile(selected_path)

# Set the parent directory
parent_directory = os.getcwd()  # Use current working directory or change as needed

# Create the dropdown widget with main and subfolders
options = get_subfolders(parent_directory)

directory_dropdown = widgets.Dropdown(
    options=options,
    description='Select Folder:',
    disabled=False,
)

# Create a button to open the selected folder
open_button = widgets.Button(description="Open Folder")
open_button.on_click(open_folder)

# Set up the observer for the dropdown
directory_dropdown.observe(on_directory_selected, names='value')

# Display the dropdown and the button
display(directory_dropdown, open_button)


Dropdown(description='Select Folder:', options=('bad_data', '----- EDF+', '-------- 440514', '----- Incomplete…

Button(description='Open Folder', style=ButtonStyle())

Selected directory: c:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\emotion_data\103918


In [9]:
muscle_threshold = 0.6  # 0.5
eog_threshold = 4  # 3
apply_proj = False  # Use same settings globally
plot_psd = True  # Generate and save PSD plots?
plot_ica_overlay = False  # Plot before and after effects of ICA cleaning
plot_topomap = True
dB = True
normalize = True
n_components = 5
output_directory = 'all_plots'
description = f'mt_{muscle_threshold}eogt_{eog_threshold}db_{dB}_nrmlizd_{normalize}_cmp_{n_components}'  # Description for output directory and file names
save_fif = True
event_list = []

import os  # Handy OS functions, explore file directory, etc.
import glob  # Useful to grab the EDF files easily
import mne  # The main EEG package / library
import matplotlib.pyplot as plt  # Use as backend when needed
from datetime import datetime  # To time & date stamp output files as needed
import re  # To sanitize filename
from mne.preprocessing import ICA  # Import it explicitly to minimize required code and refer to it more easily

# Define EEG and EOG channels
eeg_channels = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']
eog_channels = ['Fp1', 'Fp2']

# Function to sanitize filenames
def sanitize_filename(filename):
    print(f'filename: {filename}')
    return re.sub(r'[\\/*?:"<>|,]', '_', filename)

# Function to save a copy of the script
def save_script_copy(script_path, output_directory):
    sanitized_description = sanitize_filename(description)
    script_name = os.path.basename(script_path).replace('.py', f'_{sanitized_description}.py')
    output_path = os.path.join(output_directory, script_name)
    with open(script_path, 'r') as original_script:
        with open(output_path, 'w') as copy_script:
            copy_script.write(original_script.read())
    print(f"Saved a copy of the script to {output_path}")

# Function to generate plots from EDF files
def generate_plots(edf_file, output_directory):
    try:
        raw = mne.io.read_raw_edf(
            edf_file,
            misc=None,
            stim_channel=None,
            exclude=['TimestampS', 'TimestampMs', 'OrTimestampS', 'OrTimestampMs', 'Counter', 'Interpolated', 
                     'HighBitFlex', 'SaturationFlag', 'RawCq', 'Battery', 'BatteryPercent', 'MarkerHardware'],
            preload=True,
            infer_types=True,
            verbose=False
        )
        
        raw.pick(eeg_channels)  # Pick only EEG channels
        montage = mne.channels.make_standard_montage('standard_1020')  # Define the electrode locations
        raw.set_montage(montage, on_missing='ignore')
        
        raw.set_eeg_reference(ref_channels="average", projection=apply_proj)  # Set EEG average reference
        
        if apply_proj:
            raw.apply_proj()
        
        raw.filter(l_freq=1.0, h_freq=40, picks=eeg_channels, n_jobs=4)  # Apply bandpass filter
        
        # Set up ICA
        ica = ICA(n_components=n_components, random_state=97, max_iter=800)
        ica.fit(raw, picks=eeg_channels)
        
        # Find EOG and muscle artifacts
        eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=eog_channels, threshold=eog_threshold)
        muscle_noise_indices, muscle_noise_scores = ica.find_bads_muscle(raw, threshold=muscle_threshold)
        
        subfolder_name = os.path.basename(edf_file)[:6]
        subfolder_path = os.path.join(output_directory, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        # Exclude the identified artifact components
        ica.exclude = list(set(eog_indices + muscle_noise_indices))
        
        # Apply ICA to the raw data
        raw_clean = ica.apply(raw.copy())
        events, event_dict = mne.events_from_annotations(raw_clean)  # Extract all events

        # Save FIF file if required
        if save_fif:
            fif_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_raw.fif"
            fif_output_path = os.path.join(subfolder_path, fif_output_filename)
            raw_clean.save(fif_output_path, overwrite=True)

        # Create a list to hold event information
        event_list = []

        # Loop through each event to populate the event_list
        for i, event in enumerate(events):
            event_id = event[-1]
            event_name = list(event_dict.keys())[list(event_dict.values()).index(event_id)]
            event_name = event_name.split(',')[0]  # Get just the first word
            start = event[0] / raw.info['sfreq'] + 15  # Start time

            # Determine the stop event name and time
            if i + 1 < len(events):
                stop_event_id = events[i + 1][-1]
                stop_event_name = list(event_dict.keys())[list(event_dict.values()).index(stop_event_id)]
                stop_event_name = stop_event_name.split(',')[0]
                stop = events[i + 1][0] / raw.info['sfreq']  # Stop time
            else:
                stop_event_name = "End of file"  # Handle last event case
                stop = raw.times[-1]  # Set stop to the end of the file

            # Append to the event_list
            event_list.append({
                'start_event_name': event_name,
                'stop_event_name': stop_event_name,
                'start': start,
                'stop': stop
            })

        # Print out the results
        for event in event_list:
            print(f"Start Event: {event['start_event_name']}, Stop Event: {event['stop_event_name']}, Start: {event['start']:.2f}s, Stop: {event['stop']:.2f}s")

        # Loop through each event and plot PSD
        for i, event in enumerate(event_list):
            sanitized_event_name = sanitize_filename(event['start_event_name'])
            start = event['start']
            stop = event['stop']
            
            try:
                cropped_raw = raw_clean.copy().crop(tmin=start, tmax=stop)  # Crop the raw data to the event span
            except:
                cropped_raw = raw_clean.copy().crop(tmin=start)

            # Save PSD plot
            psd_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_epoch_{i + 1}_{sanitized_event_name}_psd.png"
            psd_output_path = os.path.join(subfolder_path, psd_output_filename)
            psd_fig = cropped_raw.compute_psd(picks=eeg_channels, fmin=1, fmax=40).plot(dB=dB, show=False)
            psd_fig.savefig(psd_output_path)
            plt.close(psd_fig)  # Close the figure to free up memory
            print(f"Saved PSD plot for epoch {i + 1} ({sanitized_event_name}) of {edf_file} to {psd_output_path}")

            # Save ICA overlay plot if required
            if plot_ica_overlay:
                ica_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_epoch_{i + 1}_{sanitized_event_name}_ica_overlay.png"
                ica_output_path = os.path.join(subfolder_path, ica_output_filename)
                ica_fig = ica.plot_overlay(cropped_raw, picks=eeg_channels, start=start, stop=stop, title=sanitized_event_name, show=False)
                ica_fig.savefig(ica_output_path)
                plt.close(ica_fig)  # Close the figure to free up memory

            # Save topographic map plot
            topo_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_epoch_{i + 1}_{sanitized_event_name}_psd_topomap.png"
            topo_output_path = os.path.join(subfolder_path, topo_output_filename)
            topo_fig = cropped_raw.compute_psd(picks=eeg_channels, fmin=1, fmax=40).plot_topomap(ch_type="eeg", normalize=normalize, show=False)
            plt.title(sanitized_event_name)
            topo_fig.savefig(topo_output_path)
            plt.close(topo_fig)  # Close the figure to free up memory
            print(f"Saved topomap for epoch {i + 1} ({sanitized_event_name}) of {edf_file} to {topo_output_path}")

    except Exception as e:
        print(f"Error processing {edf_file}: {e}")

# Function to find EDF files in the directory
def find_edf_files(directory):
    return glob.glob(os.path.join(directory, "*.edf"))

# Main script
if __name__ == "__main__":
    # Create output directory if it does not exist
    os.makedirs(output_directory, exist_ok=True)

    # Save a copy of the script
    # script_path = __file__  # Path to this script
    # save_script_copy(script_path, output_directory)

    # Find all EDF files in the current directory
    edf_files = find_edf_files(r'C:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\emotion_data\103918')
    print(f"Found {len(edf_files)} EDF files in the current directory.")

    # Generate plots for each EDF file found
    for edf_file in edf_files:
        print(f"Processing {edf_file}...")
        generate_plots(edf_file, output_directory)


Found 1 EDF files in the current directory.
Processing C:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\emotion_data\103918\103918.edf...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 423 samples (3.305 s)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting ICA to data using 32 channels (please be patient, this may take a while)


[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    0.5s finished


Selecting by number: 5 components
Fitting ICA took 0.4s.
Using EOG channels: Fp1, Fp2
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 1280 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper trans

In [13]:
muscle_threshold = 0.6  # 0.5
eog_threshold = 4  # 3
apply_proj = False  # Use same settings globally
plot_psd = True  # Generate and save PSD plots?
plot_ica_overlay = False  # Plot before and after effects of ICA cleaning
plot_topomap = True
dB = True
normalize = True
n_components = 5
output_directory = 'all_plots'
description = f'mt_{muscle_threshold}eogt_{eog_threshold}db_{dB}_nrmlizd_{normalize}_cmp_{n_components}'  # Description for output directory and file names
save_fif = True
event_list = []

import os  # Handy OS functions, explore file directory, etc.
import glob  # Useful to grab the EDF files easily
import mne  # The main EEG package / library
import matplotlib.pyplot as plt  # Use as backend when needed
from datetime import datetime  # To time & date stamp output files as needed
import re  # To sanitize filename
from mne.preprocessing import ICA  # Import it explicitly to minimize required code and refer to it more easily

# Define EEG and EOG channels
eeg_channels = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']
eog_channels = ['Fp1', 'Fp2']

# Function to sanitize filenames
def sanitize_filename(filename):
    print(f'filename: {filename}')
    return re.sub(r'[\\/*?:"<>|,]', '_', filename)

# Function to save a copy of the script
def save_script_copy(script_path, output_directory):
    sanitized_description = sanitize_filename(description)
    script_name = os.path.basename(script_path).replace('.py', f'_{sanitized_description}.py')
    output_path = os.path.join(output_directory, script_name)
    with open(script_path, 'r') as original_script:
        with open(output_path, 'w') as copy_script:
            copy_script.write(original_script.read())
    print(f"Saved a copy of the script to {output_path}")

# Function to generate plots from EDF files
def generate_plots(edf_file, output_directory):
    try:
        raw = mne.io.read_raw_edf(
            edf_file,
            misc=None,
            stim_channel=None,
            exclude=['TimestampS', 'TimestampMs', 'OrTimestampS', 'OrTimestampMs', 'Counter', 'Interpolated', 
                     'HighBitFlex', 'SaturationFlag', 'RawCq', 'Battery', 'BatteryPercent', 'MarkerHardware'],
            preload=True,
            infer_types=True,
            verbose=False
        )
        
        raw.pick(eeg_channels)  # Pick only EEG channels
        montage = mne.channels.make_standard_montage('standard_1020')  # Define the electrode locations
        raw.set_montage(montage, on_missing='ignore')
        
        raw.set_eeg_reference(ref_channels="average", projection=apply_proj)  # Set EEG average reference
        
        if apply_proj:
            raw.apply_proj()
        
        raw.filter(l_freq=1.0, h_freq=40, picks=eeg_channels, n_jobs=4)  # Apply bandpass filter
        
        # Set up ICA
        ica = ICA(n_components=n_components, random_state=97, max_iter=800)
        ica.fit(raw, picks=eeg_channels)
        
        # Find EOG and muscle artifacts
        eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=eog_channels, threshold=eog_threshold)
        muscle_noise_indices, muscle_noise_scores = ica.find_bads_muscle(raw, threshold=muscle_threshold)
        
        subfolder_name = os.path.basename(edf_file)[:6]
        subfolder_path = os.path.join(output_directory, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        # Exclude the identified artifact components
        ica.exclude = list(set(eog_indices + muscle_noise_indices))
        
        # Apply ICA to the raw data
        raw_clean = ica.apply(raw.copy())
        events, event_dict = mne.events_from_annotations(raw_clean)  # Extract all events

        # Save FIF file if required
        if save_fif:
            fif_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_raw.fif"
            fif_output_path = os.path.join(subfolder_path, fif_output_filename)
            raw_clean.save(fif_output_path, overwrite=True)

        # Create a list to hold event information
        event_list = []

        # Loop through each event to populate the event_list
        for i, event in enumerate(events):
            event_id = event[-1]
            event_name = list(event_dict.keys())[list(event_dict.values()).index(event_id)]
            event_name = event_name.split(',')[0]  # Get just the first word
            start = event[0] / raw.info['sfreq'] + 15  # Start time

            # Determine the stop event name and time
            if i + 1 < len(events):
                stop_event_id = events[i + 1][-1]
                stop_event_name = list(event_dict.keys())[list(event_dict.values()).index(stop_event_id)]
                stop_event_name = stop_event_name.split(',')[0]
                stop = events[i + 1][0] / raw.info['sfreq']  # Stop time
            else:
                stop_event_name = "End of file"  # Handle last event case
                stop = raw.times[-1]  # Set stop to the end of the file

            # Append to the event_list
            event_list.append({
                'start_event_name': event_name,
                'stop_event_name': stop_event_name,
                'start': start,
                'stop': stop
            })

        # Print out the results
        for event in event_list:
            print(f"Start Event: {event['start_event_name']}, Stop Event: {event['stop_event_name']}, Start: {event['start']:.2f}s, Stop: {event['stop']:.2f}s")

        # Loop through each event and plot PSD
        for i, event in enumerate(event_list):
            start_event_name = event['start_event_name']
            end_event_name = event_list[-1]['stop_event_name']  # Last event as the end event
            sanitized_event_name = sanitize_filename(f"{start_event_name} to {end_event_name}")

            # Save PSD plot
            psd_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_{sanitized_event_name}_psd.png"
            psd_output_path = os.path.join(subfolder_path, psd_output_filename)
            psd_fig = cropped_raw.compute_psd(picks=eeg_channels, fmin=1, fmax=40).plot(dB=dB, show=False)
            psd_fig.savefig(psd_output_path)
            plt.close(psd_fig)  # Close the figure to free up memory
            print(f"Saved PSD plot for epoch {i + 1} ({sanitized_event_name}) of {edf_file} to {psd_output_path}")

            # Save topographic map plot
            topo_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_{sanitized_event_name}_topomap.png"
            topo_output_path = os.path.join(subfolder_path, topo_output_filename)
            topo_fig = cropped_raw.compute_psd(picks=eeg_channels, fmin=1, fmax=40).plot_topomap(ch_type="eeg", normalize=normalize, show=False)
            plt.title(sanitized_event_name)
            topo_fig.savefig(topo_output_path)
            plt.close(topo_fig)  # Close the figure to free up memory
            print(f"Saved topomap for epoch {i + 1} ({sanitized_event_name}) of {edf_file} to {topo_output_path}")

        # (Make sure to apply similar changes for ICA overlay plots if needed)

            # Save ICA overlay plot if required
            if plot_ica_overlay:
                ica_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_epoch_{i + 1}_{sanitized_event_name}_ica_overlay.png"
                ica_output_path = os.path.join(subfolder_path, ica_output_filename)
                ica_fig = ica.plot_overlay(cropped_raw, picks=eeg_channels, start=start, stop=stop, title=sanitized_event_name, show=False)
                ica_fig.savefig(ica_output_path)
                plt.close(ica_fig)  # Close the figure to free up memory

            # Save topographic map plot
            topo_output_filename = f"{os.path.basename(edf_file).replace('.edf', '').replace('.bdf', '')}_epoch_{i + 1}_{sanitized_event_name}_psd_topomap.png"
            topo_output_path = os.path.join(subfolder_path, topo_output_filename)
            topo_fig = cropped_raw.compute_psd(picks=eeg_channels, fmin=1, fmax=40).plot_topomap(ch_type="eeg", normalize=normalize, show=False)
            plt.title(sanitized_event_name)
            topo_fig.savefig(topo_output_path)
            plt.close(topo_fig)  # Close the figure to free up memory
            print(f"Saved topomap for epoch {i + 1} ({sanitized_event_name}) of {edf_file} to {topo_output_path}")

    except Exception as e:
        print(f"Error processing {edf_file}: {e}")

# Function to find EDF files in the directory
def find_edf_files(directory):
    return glob.glob(os.path.join(directory, "*.edf"))

# Main script
if __name__ == "__main__":
    # Create output directory if it does not exist
    os.makedirs(output_directory, exist_ok=True)

    # # Save a copy of the script
    # script_path = __file__  # Path to this script
    # save_script_copy(script_path, output_directory)

    # Find all EDF files in the current directory
    edf_files = find_edf_files(r'C:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\emotion_data\103918')
    print(f"Found {len(edf_files)} EDF files in the current directory.")

    # Generate plots for each EDF file found
    for edf_file in edf_files:
        print(f"Processing {edf_file}...")
        generate_plots(edf_file, output_directory)


Found 1 EDF files in the current directory.
Processing C:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\emotion_data\103918\103918.edf...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    0.0s finished


Selecting by number: 5 components
Fitting ICA took 0.3s.
Using EOG channels: Fp1, Fp2
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 1280 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper trans